**NB**

In [ ]:
!pip install -q pandas scikit-learn openpyxl joblib



In [ ]:
import pandas as pd

import pandas as pd
import re

# Load data
df = pd.read_excel("TrainVersion_labeled.xlsx")

# Drop missing rows
df = df.dropna(subset=['text', 'Sentiment Label'])

# Preprocessing function
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # remove links
    text = re.sub(r'\@w+|\#','', text)  # remove mentions/hashtags
    text = re.sub(r'[^A-Za-z\s]', '', text)  # remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra whitespace
    return text

# Apply cleaning
df['clean_text'] = df['text'].astype(str).apply(clean_text)

In [ ]:
df['Sentiment Label'].value_counts()

,count
Sentiment Label,
1,1510
-1,1164
0,855


In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

X = df['clean_text']
y = df['Sentiment Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)



MultinomialNB()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = nb_model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7861189801699717

Classification Report:
               precision    recall  f1-score   support

          -1       0.91      0.89      0.90       238
           0       0.89      0.34      0.49       175
           1       0.70      0.97      0.81       293

    accuracy                           0.79       706
   macro avg       0.83      0.73      0.73       706
weighted avg       0.82      0.79      0.76       706



In [ ]:
# import joblib

# joblib.dump(nb_model, 'nb_model.pkl')
# joblib.dump(vectorizer, 'vectorizer.pkl')